## 1. Importação de Bibliotecas e Configurações

In [1]:
# Importações essenciais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from scipy import stats

warnings.filterwarnings('ignore')


# Configurações de visualização
plt.rcParams['figure.figsize'] = [12, 8]
sns.set_style("whitegrid")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


## 2. Carregamento e Análise Inicial dos Dados

In [3]:
# Carregar dataset original
print("Carregando dataset de sepsis...")
df = pd.read_csv('dataset_sepsis_train.csv')

print(f"Shape do dataset: {df.shape}")
print(f"Total de registros: {len(df):,}")
# Informações básicas
print("\nInformações do Dataset:")
print(df.info())

Carregando dataset de sepsis...
Shape do dataset: (1241768, 42)
Total de registros: 1,241,768

Informações do Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1241768 entries, 0 to 1241767
Data columns (total 42 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Hour              1241768 non-null  int64  
 1   HR                1119123 non-null  float64
 2   O2Sat             1079708 non-null  float64
 3   Temp              419945 non-null   float64
 4   SBP               1060857 non-null  float64
 5   MAP               1087236 non-null  float64
 6   DBP               852691 non-null   float64
 7   Resp              1051181 non-null  float64
 8   EtCO2             46047 non-null    float64
 9   BaseExcess        67324 non-null    float64
 10  HCO3              52334 non-null    float64
 11  FiO2              103618 non-null   float64
 12  pH                86094 non-null    float64
 13  PaCO2             69132 non-

In [ ]:
# Análise dos valores missing
print("ANÁLISE DE VALORES FALTANTES:")
print("=" * 50)

missing_stats = pd.DataFrame({
    'Coluna': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percent': (df.isnull().sum() / len(df)) * 100
})

missing_stats = missing_stats[missing_stats['Missing_Count'] > 0].sort_values('Missing_Percent', ascending=False)
print(missing_stats.to_string(index=False))

# Visualização dos valores faltantes
plt.figure(figsize=(12, 8))
sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='viridis')
plt.title('Padrão de Valores Faltantes no Dataset', fontsize=16, fontweight='bold')
plt.xlabel('Colunas')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. Criação do Identificador de Paciente

In [ ]:
# Criar identificador único de paciente baseado em Gender + Age
print("👤 CRIANDO IDENTIFICADOR DE PACIENTE")
print("=" * 45)

# Verificar se temos Gender e Age
if 'Gender' in df.columns and 'Age' in df.columns:
    # Criar PATIENT_ID como combinação de Gender e Age (arredondada)
    df['Age_Rounded'] = df['Age'].round(1)  # Arredondar idade para 1 casa decimal
    df['PATIENT_ID'] = df['Gender'].astype(str) + '_' + df['Age_Rounded'].astype(str)
    
    print(f"✅ PATIENT_ID criado com sucesso!")
    print(f"👥 Total de pacientes únicos: {df['PATIENT_ID'].nunique():,}")
    print(f"📊 Registros por paciente (média): {len(df) / df['PATIENT_ID'].nunique():.1f}")
    
    # Análise da distribuição de registros por paciente
    patient_counts = df['PATIENT_ID'].value_counts()
    
    print(f"\n📈 Estatísticas de registros por paciente:")
    print(f"   Mínimo: {patient_counts.min()} registros")
    print(f"   Máximo: {patient_counts.max()} registros")
    print(f"   Mediana: {patient_counts.median():.1f} registros")
    print(f"   Média: {patient_counts.mean():.1f} registros")
    
else:
    print("❌ Colunas Gender ou Age não encontradas!")
    # Criar ID sequencial como fallback
    df['PATIENT_ID'] = range(len(df))
    print("🔄 Usando ID sequencial como fallback")

In [ ]:
# Análise da distribuição por gênero e idade
if 'Gender' in df.columns:
    print("\n👥 ANÁLISE DEMOGRÁFICA:")
    print("=" * 25)
    
    # Distribuição por gênero
    gender_dist = df['Gender'].value_counts()
    print("Distribuição por Gênero:")
    for gender, count in gender_dist.items():
        percentage = (count / len(df)) * 100
        print(f"  {gender}: {count:,} ({percentage:.1f}%)")
    
    # Estatísticas de idade
    if 'Age' in df.columns:
        print(f"\n📊 Estatísticas de Idade:")
        print(f"   Mínima: {df['Age'].min():.1f} anos")
        print(f"   Máxima: {df['Age'].max():.1f} anos")
        print(f"   Média: {df['Age'].mean():.1f} anos")
        print(f"   Mediana: {df['Age'].median():.1f} anos")
        
        # Visualização da distribuição de idade por gênero
        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 2, 1)
        sns.histplot(data=df, x='Age', hue='Gender', bins=30, alpha=0.7)
        plt.title('Distribuição de Idade por Gênero', fontweight='bold')
        plt.xlabel('Idade')
        plt.ylabel('Frequência')
        
        plt.subplot(1, 2, 2)
        sns.boxplot(data=df, x='Gender', y='Age')
        plt.title('Boxplot de Idade por Gênero', fontweight='bold')
        plt.ylabel('Idade')
        
        plt.tight_layout()
        plt.show()

## 4. Análise da Variável Target

In [ ]:
# Análise da variável target (SepsisLabel)
print("🎯 ANÁLISE DA VARIÁVEL TARGET (SepsisLabel)")
print("=" * 45)

if 'SepsisLabel' in df.columns:
    target_dist = df['SepsisLabel'].value_counts().sort_index()
    
    print("Distribuição da Variável Target:")
    for label, count in target_dist.items():
        percentage = (count / len(df)) * 100
        status = "Sem Sepsis" if label == 0 else "Com Sepsis"
        print(f"  {status} ({label}): {count:,} ({percentage:.1f}%)")
    
    # Calcular taxa de desbalanceamento
    imbalance_ratio = target_dist.max() / target_dist.min()
    print(f"\n⚖️  Taxa de Desbalanceamento: {imbalance_ratio:.1f}:1")
    
    # Análise por paciente único
    patient_sepsis = df.groupby('PATIENT_ID')['SepsisLabel'].max()  # Se teve sepsis em algum momento
    patient_target_dist = patient_sepsis.value_counts().sort_index()
    
    print(f"\n👥 Distribuição por Paciente Único:")
    for label, count in patient_target_dist.items():
        percentage = (count / len(patient_sepsis)) * 100
        status = "Nunca teve Sepsis" if label == 0 else "Teve Sepsis"
        print(f"  {status}: {count:,} pacientes ({percentage:.1f}%)")
    
    # Visualização
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Gráfico 1: Distribuição geral
    target_dist.plot(kind='bar', ax=axes[0], color=['lightblue', 'lightcoral'], alpha=0.8)
    axes[0].set_title('Distribuição Geral - SepsisLabel', fontweight='bold')
    axes[0].set_xlabel('SepsisLabel')
    axes[0].set_ylabel('Contagem')
    axes[0].set_xticklabels(['Sem Sepsis', 'Com Sepsis'], rotation=0)
    axes[0].grid(True, alpha=0.3)
    
    # Gráfico 2: Distribuição por paciente
    patient_target_dist.plot(kind='bar', ax=axes[1], color=['lightgreen', 'salmon'], alpha=0.8)
    axes[1].set_title('Distribuição por Paciente Único', fontweight='bold')
    axes[1].set_xlabel('Status de Sepsis')
    axes[1].set_ylabel('Número de Pacientes')
    axes[1].set_xticklabels(['Nunca teve', 'Teve Sepsis'], rotation=0)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ Coluna SepsisLabel não encontrada!")

## 5. Imputação Inteligente por Paciente

In [ ]:
# Identificar colunas numéricas para imputação (excluindo ID e target)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cols_to_exclude = ['PATIENT_ID', 'SepsisLabel', 'Age_Rounded']
numeric_cols = [col for col in numeric_cols if col not in cols_to_exclude]

print("🔄 IMPUTAÇÃO INTELIGENTE POR PACIENTE")
print("=" * 40)
print(f"📊 Colunas numéricas para imputação: {len(numeric_cols)}")
print(f"📋 Colunas: {numeric_cols[:5]}..." if len(numeric_cols) > 5 else f"📋 Colunas: {numeric_cols}")

# Criar cópia do dataframe para imputação
df_imputed = df.copy()

# Função de imputação por paciente
def impute_patient_data(patient_data):
    """
    Aplica imputação inteligente para um paciente específico:
    1. Forward fill (propagar último valor válido)
    2. Backward fill (propagar próximo valor válido)
    3. Média do paciente para valores ainda faltantes
    """
    patient_imputed = patient_data.copy()
    
    for col in numeric_cols:
        if col in patient_imputed.columns:
            # 1. Forward fill
            patient_imputed[col] = patient_imputed[col].fillna(method='ffill')
            
            # 2. Backward fill
            patient_imputed[col] = patient_imputed[col].fillna(method='bfill')
            
            # 3. Média do paciente (se ainda houver NaN)
            if patient_imputed[col].isnull().any():
                patient_mean = patient_imputed[col].mean()
                if not pd.isna(patient_mean):
                    patient_imputed[col] = patient_imputed[col].fillna(patient_mean)
    
    return patient_imputed

# Aplicar imputação por paciente
print("\n🔄 Aplicando imputação por paciente...")

imputed_patients = []
total_patients = df_imputed['PATIENT_ID'].nunique()

for i, (patient_id, patient_data) in enumerate(df_imputed.groupby('PATIENT_ID')):
    if i % 1000 == 0:  # Progress indicator
        print(f"   Processando paciente {i+1}/{total_patients} ({(i+1)/total_patients*100:.1f}%)")
    
    imputed_patient = impute_patient_data(patient_data)
    imputed_patients.append(imputed_patient)

# Combinar todos os pacientes imputados
df_imputed = pd.concat(imputed_patients, ignore_index=True)

print(f"✅ Imputação por paciente concluída!")

In [ ]:
# Comparar missing values antes e depois da imputação por paciente
print("📊 COMPARAÇÃO: ANTES vs DEPOIS DA IMPUTAÇÃO POR PACIENTE")
print("=" * 60)

missing_before = df[numeric_cols].isnull().sum()
missing_after = df_imputed[numeric_cols].isnull().sum()

comparison = pd.DataFrame({
    'Coluna': numeric_cols,
    'Missing_Antes': missing_before.values,
    'Missing_Depois': missing_after.values,
    'Redução': missing_before.values - missing_after.values,
    'Redução_%': ((missing_before.values - missing_after.values) / missing_before.values * 100).round(1)
})

# Mostrar apenas colunas que tinham missing values
comparison_filtered = comparison[comparison['Missing_Antes'] > 0].sort_values('Redução_%', ascending=False)

if len(comparison_filtered) > 0:
    print(comparison_filtered.to_string(index=False))
    
    total_missing_before = missing_before.sum()
    total_missing_after = missing_after.sum()
    total_reduction = ((total_missing_before - total_missing_after) / total_missing_before * 100)
    
    print(f"\n📈 RESUMO GERAL:")
    print(f"   Total missing antes: {total_missing_before:,}")
    print(f"   Total missing depois: {total_missing_after:,}")
    print(f"   Redução total: {total_reduction:.1f}%")
else:
    print("✅ Nenhum valor faltante encontrado nas colunas numéricas!")

## 6. Imputação Global para Valores Remanescentes

In [ ]:
# Para valores que ainda estão faltando, usar imputação global
print("🌐 IMPUTAÇÃO GLOBAL PARA VALORES REMANESCENTES")
print("=" * 48)

# Verificar se ainda há valores faltantes
remaining_missing = df_imputed[numeric_cols].isnull().sum()
cols_with_missing = remaining_missing[remaining_missing > 0]

if len(cols_with_missing) > 0:
    print(f"📋 Colunas com valores ainda faltantes: {len(cols_with_missing)}")
    print(cols_with_missing.to_string())
    
    # Estratégias de imputação global
    print("\n🔄 Aplicando estratégias de imputação global...")
    
    for col in cols_with_missing.index:
        missing_count = cols_with_missing[col]
        
        # Estratégia 1: Mediana por gênero (se disponível)
        if 'Gender' in df_imputed.columns:
            gender_medians = df_imputed.groupby('Gender')[col].median()
            
            for gender in gender_medians.index:
                mask = (df_imputed['Gender'] == gender) & (df_imputed[col].isnull())
                df_imputed.loc[mask, col] = gender_medians[gender]
        
        # Estratégia 2: Mediana global para valores ainda faltantes
        global_median = df_imputed[col].median()
        df_imputed[col] = df_imputed[col].fillna(global_median)
        
        print(f"   ✅ {col}: {missing_count} valores imputados")
    
    # Verificação final
    final_missing = df_imputed[numeric_cols].isnull().sum().sum()
    print(f"\n📊 Valores faltantes após imputação global: {final_missing}")
    
else:
    print("✅ Nenhum valor faltante remanescente - imputação por paciente foi suficiente!")

## 7. Feature Engineering

In [ ]:
# Feature Engineering: Criar features derivadas
print("⚙️  FEATURE ENGINEERING")
print("=" * 25)

df_engineered = df_imputed.copy()

# 1. Estatísticas por paciente
print("📊 Criando estatísticas por paciente...")

patient_stats = df_engineered.groupby('PATIENT_ID')[numeric_cols].agg({
    col: ['mean', 'std', 'min', 'max', 'count'] for col in numeric_cols
})

# Flatten column names
patient_stats.columns = ['_'.join(col).strip() for col in patient_stats.columns]

# Merge back to original dataframe
df_engineered = df_engineered.merge(patient_stats, on='PATIENT_ID', how='left')

print(f"   ✅ {len(patient_stats.columns)} estatísticas por paciente criadas")

# 2. Features de tendência (se há múltiplas medições por paciente)
print("\n📈 Criando features de tendência...")

def calculate_trends(patient_data):
    """Calcula tendências para um paciente"""
    trends = {}
    
    for col in numeric_cols[:5]:  # Limitar para evitar muitas features
        if col in patient_data.columns and len(patient_data) > 1:
            values = patient_data[col].dropna()
            if len(values) > 1:
                # Tendência linear simples (diferença entre último e primeiro valor)
                trend = values.iloc[-1] - values.iloc[0]
                trends[f'{col}_trend'] = trend
            else:
                trends[f'{col}_trend'] = 0
        else:
            trends[f'{col}_trend'] = 0
    
    return pd.Series(trends)

# Calcular tendências por paciente
patient_trends = df_engineered.groupby('PATIENT_ID').apply(calculate_trends)
patient_trends = patient_trends.reset_index()

# Merge trends back
df_engineered = df_engineered.merge(patient_trends, on='PATIENT_ID', how='left')

print(f"   ✅ {len([col for col in patient_trends.columns if '_trend' in col])} features de tendência criadas")

# 3. Features de risco (baseadas em conhecimento médico)
print("\n🏥 Criando features de risco médico...")

# Idade como fator de risco
if 'Age' in df_engineered.columns:
    df_engineered['Age_Risk'] = (df_engineered['Age'] > 65).astype(int)
    df_engineered['Age_Group'] = pd.cut(df_engineered['Age'], 
                                       bins=[0, 18, 35, 50, 65, 100], 
                                       labels=['Child', 'Young_Adult', 'Adult', 'Middle_Age', 'Senior'])
    
# Combinar sinais vitais (se disponíveis)
vital_signs = ['HR', 'Temp', 'SBP', 'MAP', 'Resp']  # Comuns em dados de sepsis
available_vitals = [col for col in vital_signs if col in df_engineered.columns]

if len(available_vitals) >= 2:
    # Score de instabilidade (soma dos z-scores dos sinais vitais)
    scaler = StandardScaler()
    vitals_scaled = scaler.fit_transform(df_engineered[available_vitals])
    df_engineered['Vitals_Instability_Score'] = np.abs(vitals_scaled).sum(axis=1)
    
    print(f"   ✅ Score de instabilidade criado usando {len(available_vitals)} sinais vitais")

print(f"\n📊 Shape final após Feature Engineering: {df_engineered.shape}")
print(f"📈 Novas features criadas: {df_engineered.shape[1] - df_imputed.shape[1]}")

## 8. Detecção e Tratamento de Outliers

In [ ]:
# Detecção de outliers usando IQR
print("🔍 DETECÇÃO E TRATAMENTO DE OUTLIERS")
print("=" * 40)

df_no_outliers = df_engineered.copy()

# Selecionar colunas numéricas originais para análise de outliers
original_numeric_cols = [col for col in numeric_cols if col in df_no_outliers.columns]

def detect_outliers_iqr(data, column, multiplier=1.5):
    """Detecta outliers usando método IQR"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    
    outliers = (data[column] < lower_bound) | (data[column] > upper_bound)
    return outliers

outlier_summary = []

for col in original_numeric_cols[:10]:  # Analisar primeiras 10 colunas
    outliers = detect_outliers_iqr(df_no_outliers, col)
    outlier_count = outliers.sum()
    outlier_percent = (outlier_count / len(df_no_outliers)) * 100
    
    outlier_summary.append({
        'Coluna': col,
        'Outliers': outlier_count,
        'Porcentagem': outlier_percent
    })
    
    # Tratar outliers usando capping (winsorizing)
    if outlier_percent > 0.5 and outlier_percent < 10:  # Só tratar se entre 0.5% e 10%
        Q1 = df_no_outliers[col].quantile(0.25)
        Q3 = df_no_outliers[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Capping
        df_no_outliers[col] = df_no_outliers[col].clip(lower=lower_bound, upper=upper_bound)
        
        print(f"   🔧 {col}: {outlier_count} outliers ({outlier_percent:.1f}%) tratados por capping")

# Resumo de outliers
outlier_df = pd.DataFrame(outlier_summary)
outlier_df = outlier_df[outlier_df['Outliers'] > 0].sort_values('Porcentagem', ascending=False)

if len(outlier_df) > 0:
    print("\n📊 Resumo de Outliers Detectados:")
    print(outlier_df.to_string(index=False, float_format='%.2f'))
else:
    print("✅ Nenhum outlier significativo detectado!")

## 9. Normalização e Escalonamento

In [ ]:
# Normalização dos dados
print("📏 NORMALIZAÇÃO E ESCALONAMENTO")
print("=" * 35)

df_scaled = df_no_outliers.copy()

# Separar features numéricas para escalonamento
# Excluir colunas categóricas e ID
columns_to_exclude = ['PATIENT_ID', 'SepsisLabel', 'Gender', 'Age_Group', 'Age_Rounded']
numeric_features = df_scaled.select_dtypes(include=[np.number]).columns.tolist()
numeric_features = [col for col in numeric_features if col not in columns_to_exclude]

print(f"📊 Colunas numéricas para escalonamento: {len(numeric_features)}")

# Usar RobustScaler (menos sensível a outliers)
scaler = RobustScaler()

# Aplicar escalonamento
df_scaled[numeric_features] = scaler.fit_transform(df_scaled[numeric_features])

print(f"✅ Escalonamento aplicado usando RobustScaler")
print(f"📈 Features escalonadas: {len(numeric_features)}")

# Verificar distribuição após escalonamento
print("\n📊 Estatísticas após escalonamento (primeiras 5 colunas):")
print(df_scaled[numeric_features[:5]].describe().round(3))

## 10. Divisão dos Dados

In [ ]:
# Divisão estratificada dos dados
print("🔀 DIVISÃO DOS DADOS")
print("=" * 20)

# Preparar features e target
if 'SepsisLabel' in df_scaled.columns:
    # Remover colunas não-features
    feature_columns = [col for col in df_scaled.columns 
                      if col not in ['SepsisLabel', 'PATIENT_ID', 'Age_Rounded']]
    
    X = df_scaled[feature_columns]
    y = df_scaled['SepsisLabel']
    
    print(f"📊 Shape das features (X): {X.shape}")
    print(f"🎯 Shape do target (y): {y.shape}")
    
    # Divisão estratificada
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, 
        stratify=y, 
        random_state=42
    )
    
    print(f"\n📈 Divisão realizada:")
    print(f"   Treino: {X_train.shape[0]:,} amostras ({X_train.shape[0]/len(X)*100:.1f}%)")
    print(f"   Teste:  {X_test.shape[0]:,} amostras ({X_test.shape[0]/len(X)*100:.1f}%)")
    
    # Verificar balanceamento
    train_dist = y_train.value_counts(normalize=True).sort_index()
    test_dist = y_test.value_counts(normalize=True).sort_index()
    
    print(f"\n⚖️  Distribuição do target:")
    print(f"   Treino - Sem Sepsis: {train_dist[0]:.1%}, Com Sepsis: {train_dist[1]:.1%}")
    print(f"   Teste  - Sem Sepsis: {test_dist[0]:.1%}, Com Sepsis: {test_dist[1]:.1%}")
    
else:
    print("❌ Coluna SepsisLabel não encontrada para divisão!")

## 11. Salvamento dos Dados Processados

In [ ]:
# Salvar datasets processados
print("💾 SALVAMENTO DOS DADOS PROCESSADOS")
print("=" * 38)

# Salvar dataset completo processado
output_file = 'dataset_sepsis_prepared_v2.csv'
df_scaled.to_csv(output_file, index=False)
print(f"✅ Dataset completo salvo: {output_file}")
print(f"   Shape: {df_scaled.shape}")

if 'SepsisLabel' in df_scaled.columns:
    # Salvar conjuntos de treino e teste
    train_data = pd.concat([X_train, y_train], axis=1)
    test_data = pd.concat([X_test, y_test], axis=1)
    
    train_file = 'dataset_sepsis_train_v2.csv'
    test_file = 'dataset_sepsis_test_v2.csv'
    
    train_data.to_csv(train_file, index=False)
    test_data.to_csv(test_file, index=False)
    
    print(f"✅ Dataset de treino salvo: {train_file}")
    print(f"   Shape: {train_data.shape}")
    print(f"✅ Dataset de teste salvo: {test_file}")
    print(f"   Shape: {test_data.shape}")
    
    # Salvar informações sobre as features
    feature_info = pd.DataFrame({
        'Feature': feature_columns,
        'Type': [str(X[col].dtype) for col in feature_columns],
        'Missing_Original': [df[col].isnull().sum() if col in df.columns else 0 for col in feature_columns],
        'Missing_Final': [df_scaled[col].isnull().sum() for col in feature_columns]
    })
    
    feature_info.to_csv('features_info_v2.csv', index=False)
    print(f"✅ Informações das features salvas: features_info_v2.csv")

print(f"\n🎉 PROCESSAMENTO CONCLUÍDO COM SUCESSO!")
print(f"📊 Total de features: {len(feature_columns) if 'feature_columns' in locals() else 'N/A'}")
print(f"🏥 Total de registros processados: {len(df_scaled):,}")
print(f"👥 Total de pacientes únicos: {df_scaled['PATIENT_ID'].nunique():,}")